In [1]:
from osgeo import ogr

In [19]:
path_kunming = 'E:\\YNU\\5\\OpenSourceGIS\\Act_3\\OnClass\\kunming.gpkg'
#"E:\\YNU\\5\\OpenSourceGIS\\Act_3\\OnClass\\昆明市行政区划.shp"

#"E:/YNU/5/OpenSourceGIS/Act_3/OnClass/昆明市行政区划.shp"

#"E:/YNU/5/OpenSourceGIS/Act_3/OnClass/kunming_boundary.gpkg"

#"E:/YNU/5/OpenSourceGIS/Act_3/OnClass/kunming_boundary.shp"


In [20]:
ds =ogr.Open(path_kunming,0)  # 0代表只读，1代表可写
print(ds)
layer = ds.GetLayer(0)
print(layer)
num_fea = layer.GetFeatureCount()
print(num_fea)
fea = layer.GetFeature(1) # 或1
print(fea)
num_field = fea.GetFieldCount()
print(num_field)

<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x000001982B00CF60> >
<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x000001982B337F60> >
14
OGRFeature(district):1
  dt_adcode (String) = 530102
  dt_name (String) = 五华区
  ct_adcode (String) = 530100
  ct_name (String) = 昆明市
  pr_adcode (String) = 530000
  pr_name (String) = 云南省
  cn_adcode (String) = 100000
  cn_name (String) = 中华人民共和国
  MULTIPOLYGON (((102.713306 25.136515,102.715473 25.137243,102.718014 25.132135,102.717876 25.128678,102.718119 25.123739,102.71885 25.121694,102.71912 25.121421,102.719703 25.120299,102.720528 25.119658,102.721433 25.119367,102.722876 25.119076,102.723921 25.119106,102.724629 25.119148,102.725112 25.119327,102.725587 25.119217,102.725537 25.118238,102.726277 25.118004,102.72677 25.117925,102.727328 25.117809,102.727665 25.117638,102.727768 25.117585,102.730694 25.117628,102.732115 25.119994,102.733644 25.122731,102.734408 25.123695,102.735598 25.12

In [21]:
fea = layer.GetFeature(1)
for i in range(0, num_field):
    print(i)
    field = fea.GetFieldDefnRef(i).GetName()
    print(field)

0
dt_adcode
1
dt_name
2
ct_adcode
3
ct_name
4
pr_adcode
5
pr_name
6
cn_adcode
7
cn_name


In [22]:
layer.GetFeatureCount()

14

In [23]:
field_name = 'dt_name'
fea = layer.GetFeature(14)
field_value = fea.GetField(field_name)
print(field_value)
for i in range(0, layer.GetFeatureCount()):
    fea = layer.GetFeature(i+1)
    field_value = fea.GetField(field_name)
    print(field_value)
    

安宁市
五华区
盘龙区
官渡区
西山区
东川区
呈贡区
晋宁区
富民县
宜良县
石林彝族自治县
嵩明县
禄劝彝族苗族自治县
寻甸回族彝族自治县
安宁市


In [24]:
spa_ref = layer.GetSpatialRef()
print(spa_ref)

GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4326"]]


In [25]:
fea_geo = fea.GetGeometryRef()
fea_geo_name = fea_geo.GetGeometryName()
print(fea_geo_name)

MULTIPOLYGON


In [26]:
#ds = None

1.写出只包含某一个字段的矢量数据

In [27]:
path_kunming_path = "E:\\YNU\\5\\OpenSourceGIS\\Act_3\\OnClass\\kunming_ONE_1.gpkg"

In [28]:
field_name = 'dt_name'
in_ds = ogr.Open(path_kunming, 0)
in_layer = in_ds.GetLayer(0)
print(in_layer)

<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x000001982B724660> >


In [29]:
driver = ogr.GetDriverByName('GPKG') ##'ESRI Shapefile'
ds_out = driver.CreateDataSource(path_kunming_path)
layer_out = ds_out.CreateLayer('kunming_ONE_1',
                               geom_type = ogr.wkbPolygon,
                               srs=in_layer.GetSpatialRef())##创建图层，定义成一个面,空间参考与读入数据一致
field_defn = ogr.FieldDefn(field_name,ogr.OFTString)

##写入图层
layer_out.CreateField(field_defn)
##要素定义
fea_defn = layer_out.GetLayerDefn()

##写入每个字段特定信息
for i in range(in_layer.GetFeatureCount()):
    in_fea = in_layer.GetFeature(i+1)
    in_geo = in_fea.geometry()
    ##创建一个空要素，用于写出，要素定义作为参数
    fea_out = ogr.Feature(fea_defn)
    fea_out.SetGeometry(in_geo)##设置几何
    ##设置属性值
    field_value = in_fea.GetField(field_name)##提取字段值
    fea_out.SetField(0,field_value)###写入（索引，字段值）
    ##将要素写出到图层
    layer_out.CreateFeature(fea_out)

in_ds = None
ds_out = None


2.写出只含某一要素的矢量数据。

In [30]:
path = 'E:\\YNU\\5\\OpenSourceGIS\\Act_3\\OnClass\\kunming_ONE_2.gpkg'

In [31]:
in_ds = ogr.Open(path_kunming,0)
in_layer = in_ds.GetLayer()

In [32]:
driver = ogr.GetDriverByName('GPKG')
ds_out = driver.CreateDataSource(path)
layer_out = ds_out.CreateLayer('kunming_ONE_2', 
                               geom_type=ogr.wkbPolygon,
                               srs=in_layer.GetSpatialRef())
fea_defn = layer_out.GetLayerDefn()
layer_defn = layer_out.GetLayerDefn()
print('number of fields:', layer_defn.GetFieldCount())

in_fea = in_layer.GetFeature(1)
for i in range(in_fea.GetFieldCount()):
    field_defn = in_fea.GetFieldDefnRef(i)
    layer_out.CreateField(field_defn)

layer_defn = layer_out.GetLayerDefn()
print('number of fileds:', layer_defn.GetFieldCount())

for i in range(in_layer.GetFeatureCount()):
    in_fea = in_layer.GetFeature(i+1)
    if str(in_fea.GetField('dt_name')) == '呈贡区':
        fea_out = ogr.Feature(fea_defn)
        fea_out.SetGeometry(in_fea.geometry())
        for i in range(in_fea.GetFieldCount()):
            field_value = in_fea.GetField(i)
            fea_out.SetField(i, field_value)
        layer_out.CreateFeature(fea_out)
in_ds = None
ds_out = None

number of fields: 0
number of fileds: 8


# 随机点名

In [33]:
'''
students = ['郑道旸', '孙苡皓']
len(students)
'''

"\nstudents = ['郑道旸', '孙苡皓']\nlen(students)\n"

In [34]:
'''
import numpy as np
id = np.randow.randint(0,2)
print(students[id])
'''

'\nimport numpy as np\nid = np.randow.randint(0,2)\nprint(students[id])\n'